In [10]:
import argparse
import networkx as nx
import pandas as pd
from tqdm import tqdm
from pymongo import MongoClient
from gqlalchemy import Memgraph
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from pathlib import Path

In [11]:
def get_mongo_collection(mongo_db, database_name, collection_name):
    # mongo_db = MongoClient("mongodb://localhost:27017")
    return mongo_db[database_name][collection_name]

mongo_db = MongoClient("mongodb://localhost:27017")

In [12]:
PROPERTIES = [
        "fired",
        "fired_fraction",
        "activity",
        "dampening",
        "current_calcium",
        "target_calcium",
        "synaptic_input",
        "background_input",
        "grown_axons",
        "connected_axons",
        "grown_dendrites",
        "connected_dendrites",
    ]

In [13]:
# will run tonight:

read_from_collection = get_mongo_collection(mongo_db, "calcium", "level_0_nodes")
write_to_collection = get_mongo_collection(mongo_db, "test", "test")

for current_step in tqdm(range(0, 1_000_000, 100)):
    results = list(read_from_collection.find({"step": current_step}, {"_id": 0, "neuron_id": 1, **{current_property: 1 for current_property in PROPERTIES}, }))
    results.sort(key=lambda x : x['neuron_id'])

    big_documents = []
    for current_property in PROPERTIES:
        big_documents.append({"step": current_step, "property": current_property, "values": list(map(lambda x: x[current_property], results))})
    insertion_res = write_to_collection.insert_many(big_documents)


  0%|          | 1/10000 [00:45<127:24:20, 45.87s/it]


KeyboardInterrupt: 